In [0]:
from graphframes import *
from pyspark.sql import SparkSession
import psycopg2
import pandas as pd

# access the postgresql server
db = psycopg2.connect(
    host="codd04.research.northwestern.edu",
    port = "5433",
    database="postgres",
    user="cpdbstudent",
    password="DataSci4AI")

cursor = db.cursor()

# construct nodes - officer IDs
nodes_query = "SELECT id FROM data_officer;"

cursor.execute(nodes_query)
nodes = cursor.fetchall()

df_nodes = pd.DataFrame(nodes)
colnames = [desc[0] for desc in cursor.description]
df_nodes.columns = colnames

nodes = spark.createDataFrame(df_nodes)

# construct edges - allegations involving two officers (the source and destination nodes)
edges_query = "SELECT da1.officer_id src, da2.officer_id dst FROM data_officerallegation da1 JOIN data_officerallegation da2 ON da1.allegation_id = da2.allegation_id GROUP BY da1.officer_id, da2.officer_id;"

cursor.execute(edges_query)
edges = cursor.fetchall()

df_edges = pd.DataFrame(edges)
colnames = [desc[0] for desc in cursor.description]
df_edges.columns = colnames

edges = spark.createDataFrame(df_edges)

# construct graph
graph = GraphFrame(nodes, edges)

# Question 1: PageRank algorithm
pr = graph.pageRank(resetProbability=0.15, tol=0.01)
pr.vertices.orderBy("pagerank", ascending=False).show()

# Question 2: Triangle Count algorithm
tc = graph.triangleCount()
tc.select("id", "count").orderBy("count", ascending=False).show()

data_collect = tc.collect()

sum = 0
num_vals = 0
for row in data_collect:
    sum = sum + row['count']
    num_vals = num_vals + 1
    
avg = sum / num_vals

print("Allegation graph avg: ", avg)

# Question 2 (part 2): Triangle Count algorithm on TRR graph
# construct edges - TRRs involving two officers (the source and destination nodes)
edges_query_trr = "SELECT t1.officer_id src, t2.officer_id dst FROM trr_trr t1 JOIN trr_trr t2 ON t1.event_id = t2.event_id GROUP BY t1.officer_id, t2.officer_id;"

cursor.execute(edges_query_trr)
edges_trr = cursor.fetchall()

df_edges_trr = pd.DataFrame(edges_trr)
colnames_trr = [desc[0] for desc in cursor.description]
df_edges_trr.columns = colnames_trr

edges_trr = spark.createDataFrame(df_edges_trr)

# construct graph
graph_trr = GraphFrame(nodes, edges_trr)

# run Triangle Count
tc_trr = graph_trr.triangleCount()
tc_trr.select("id", "count").orderBy("count", ascending=False).show()

data_collect_trr = tc_trr.collect()

sum_trr = 0
num_vals_trr = 0
for row in data_collect_trr:
    sum_trr = sum_trr + row['count']
    num_vals_trr = num_vals_trr + 1
    
avg_trr = sum_trr / num_vals_trr

print("TRR graph avg: ", avg_trr)

+-----+------------------+
 id| pagerank|
+-----+------------------+
 8138|13.904908099403418|
 9821|11.385383879450147|
 2375|10.804093543762958|
31906| 9.617070055644799|
21530| 9.53766008185453|
17816| 8.731868393866312|
13303| 8.660429674612164|
10442| 8.620371042162118|
 6315| 8.563685507047259|
14294| 8.42738677728057|
 3033| 8.360947899243373|
18042| 8.249267648977327|
28273| 8.225723426009008|
 441| 8.134472494398423|
 5667| 8.13333496598418|
 8562| 8.101488406803254|
16567| 8.083116184628027|
29882| 7.991070439990282|
11266|7.9326331360650775|
 9254| 7.923984903850231|
+-----+------------------+
only showing top 20 rows

+-----+-----+
 id|count|
+-----+-----+
 6315|32118|
 3033|32117|
 3744|32073|
18042|27855|
 441|27823|
21530|23900|
27349|23518|
 5180|23499|
 5667|23487|
16747|23477|
 8844|23475|
14750|23472|
23654|23472|
19856|20185|
 8138|19322|
29882|18773|
28273|18648|
28459|18602|
 5577|18539|
30841|18502|
+-----+-----+
only showing top 20 rows

Allegation graph avg: 246.5333239555493
+-----+-----+
 id|count|
+-----+-----+
21371| 206|
13313| 182|
11615| 180|
30561| 162|
10583| 148|
32118| 143|
31337| 137|
32271| 129|
 8374| 115|
 8628| 114|
20154| 113|
 3558| 110|
32175| 108|
29454| 105|
 8138| 103|
20860| 102|
26018| 102|
 5204| 101|
31576| 99|
27900| 99|
+-----+-----+
only showing top 20 rows

TRR graph avg: 1.794007596005064